In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import math
import sqlite3
import pandas as pd
import numpy as np
import urllib
from lxml import etree
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sqlite3
import pickle
import math
import numpy as np

## Scraping Individual and Team Data From Basketball Reference

In [2]:
#initial request
url = 'https://www.basketball-reference.com/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
#find links for individual page
urls = []
for i in soup.find_all('tr', class_="full_table"):
    urls.append(url + i.find('a', href=True)['href'])
urls

['https://www.basketball-reference.com//teams/MIL/2020.html',
 'https://www.basketball-reference.com//teams/TOR/2020.html',
 'https://www.basketball-reference.com//teams/BOS/2020.html',
 'https://www.basketball-reference.com//teams/MIA/2020.html',
 'https://www.basketball-reference.com//teams/IND/2020.html',
 'https://www.basketball-reference.com//teams/PHI/2020.html',
 'https://www.basketball-reference.com//teams/BRK/2020.html',
 'https://www.basketball-reference.com//teams/ORL/2020.html',
 'https://www.basketball-reference.com//teams/WAS/2020.html',
 'https://www.basketball-reference.com//teams/CHO/2020.html',
 'https://www.basketball-reference.com//teams/CHI/2020.html',
 'https://www.basketball-reference.com//teams/NYK/2020.html',
 'https://www.basketball-reference.com//teams/DET/2020.html',
 'https://www.basketball-reference.com//teams/ATL/2020.html',
 'https://www.basketball-reference.com//teams/CLE/2020.html',
 'https://www.basketball-reference.com//teams/LAL/2020.html',
 'https:

In [4]:
with open('listofurls.pkl', 'wb') as f:
    pickle.dump(urls, f)

In [5]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")
driver = webdriver.Chrome(executable_path='/Users/Anubhav/Desktop/Flatiron School/QCstats/chromedriver', options=option)

In [6]:
all_individuals = pd.DataFrame()
all_teamstats = pd.DataFrame()
for url in range(len(urls)):
    #go to team site
    driver.get(urls[url])
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "all_roster")))

    #create dataframes for each table
    df0 = pd.read_html(driver.find_element_by_css_selector('table[id="per_game"]').get_attribute('outerHTML'))[0]
    df1 = pd.read_html(driver.find_element_by_css_selector('table[id="totals"]').get_attribute('outerHTML'))[0]
    df2 = pd.read_html(driver.find_element_by_css_selector('table[id="per_minute"]').get_attribute('outerHTML'))[0]
    df3 = pd.read_html(driver.find_element_by_css_selector('table[id="per_poss"]').get_attribute('outerHTML'))[0]
    df4 = pd.read_html(driver.find_element_by_css_selector('table[id="advanced"]').get_attribute('outerHTML'))[0]
    df5 = pd.read_html(driver.find_element_by_css_selector('table[id="shooting"]').get_attribute('outerHTML'))[0]
    df6 = pd.read_html(driver.find_element_by_css_selector('table[id="pbp"]').get_attribute('outerHTML'))[0] 
    df7 = pd.read_html(driver.find_element_by_css_selector('table[id="salaries2"]').get_attribute('outerHTML'))[0]
    df8 = pd.read_html(driver.find_element_by_css_selector('table[id="roster"]').get_attribute('outerHTML'))[0]

    #change column names of all player tables
    df0.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/G', 'FGMade/G', 'FGA/G', 'FGPerc',
                   'ThreeMade/G', 'ThreeA/G', 'ThreePerc','TwoMade/G', 'TwoA/G',
                   'TwoPerc', 'eFGPerc', 'FTMade/G', 'FTA/G', 'FTPerc', 'ORB/G',
                   'DRB/G', 'TRB/G','AST/G', 'STL/G', 'BLK/G', 'TOV/G', 'PF/G', 'PTS/G']
    df1.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MPTotal', 'FGMadeTotal',
                   'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal', 'ThreePerc',
                   'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'eFGPerc', 'FTMadeTotal',
                   'FTATotal', 'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal',
                   'ASTTotal', 'STLTotal', 'BLKTotal', 'TOVTotal', 'PFTotal', 'PTSTotal']
    df1.drop(df1.tail(1).index, inplace=True)
    df2.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/THREESIX', 'FGMade/THREESIX',
                   'FGA/THREESIX', 'FGPerc', 'ThreeMade/THREESIX', 'ThreeA/THREESIX',
                   'ThreePerc','TwoMade/THREESIX', 'TwoA/THREESIX', 'TwoPerc',
                   'FTMade/THREESIX', 'FTA/THREESIX', 'FTPerc', 'ORB/THREESIX', 'DRB/THREESIX',
                   'TRB/THREESIX', 'AST/THREESIX', 'STL/THREESIX', 'BLK/THREESIX',
                   'TOV/THREESIX', 'PF/THREESIX', 'PTS/THREESIX']
    df3.columns = ['Rk', 'Name', 'Age', 'G', 'GS', 'MP/Hund', 'FGMade/Hund', 'FGA/Hund',
                   'FGPerc', 'ThreeMade/Hund', 'ThreeA/Hund', 'ThreePerc',
                   'TwoMade/Hund', 'TwoA/Hund', 'TwoPerc', 'FTMade/Hund', 'FTA/Hund',
                   'FTPerc', 'ORB/Hund', 'DRB/Hund', 'TRB/Hund', 'AST/Hund', 'STL/Hund',
                   'BLK/Hund', 'TOV/Hund', 'PF/Hund', 'PTS/Hund', 'Blank', 'ORTG', 'DRTG']
    df4.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'PER', 'TSPerc', 'PercThreeA',
                   'PercFTA/FG', 'PercORB', 'PercDRB', 'PercTRB', 'PercAST', 'PercSTL',
                   'PercBLK', 'PercTOV', 'USG', 'Blank1', 'OWS', 'DWS', 'WS', 'WS/48',
                   'Blank2', 'OBPM', 'DBPM', 'BPM', 'VORP']
    df5.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'FGPerc', 'AvgDis',
                   'PercTwoA', 'PercZeroThreefeetA','PercThreefeetTenA', 'PercTenSixTA',
                   'PercSixTThreeA', 'PercThreeA', 'TwoPerc', 'ZeroThreePerc',
                   'ThreefeetTenPerc', 'TenSixTPerc', 'SixTThreePerc', 'ThreePerc',
                   'PercTwoAstd', 'PercFGADunks', 'MadeDunks', 'PercThreeAstd',
                   'Perc3PACorner', 'ThreePercCorner', 'HeavesA', 'MadeHeaves']
    df6.columns = ['Rk', 'Name', 'Age', 'G', 'MPTotal', 'PercasPG', 'PercasSG',
                   'PercasSF', 'PercasPF', 'PercasC', 'PMP100OnCourt', 'NPMP100',
                   'TOVBadPass', 'TOVLostBall', 'ShootFoulsC', 'OFoulsC',
                   'ShootFoulsD', 'OFoulsD', 'PGA', 'FAAnd1', 'FABlkd']
    df7.columns = ['Rk', 'Name', 'Salary']
    df7.Salary = df7.Salary.str.replace(',', '').str[1:]
    df7.drop_duplicates(subset='Name', keep='first',inplace=True)
    df8.columns = ['No', 'Name', 'Pos', 'Ht', 'Wt', 'Birth Date', 'Country', 'Exp', 'College']
    df8.Ht = (pd.to_numeric(df8.Ht.str.split('-').str[0])*12) + pd.to_numeric(df8.Ht.str.split('-').str[1])
    df8.drop(['No', 'Pos', 'Wt', 'Birth Date', 'Country', 'College'], axis = 1, inplace = True)
    df8.drop_duplicates(subset='Name', keep='first',inplace=True)
    
    #new
    droplist = []
    for name in df8.Name:
        if 'TW' in name:
            droplist.append(name.split()[0] + ' ' + name.split()[1])
        else:
            pass
        
    for table in range(0,9):
        exec(f"df{table}.Name = df{table}.Name.str.split().str[0] + ' ' + df{table}.Name.str.split().str[1]")

    #drop uncessary columns from each dataframe
    for table in range(1,9):
        checklist = []
        for x in range(0,table):
            exec(f"checklist.extend(list(df{x}.columns))")
            checklist.remove('Name')
        exec(f"df{table}.drop([col for col in df{table}.columns if col in checklist], axis = 1, inplace = True)")
        exec(f"blank = [col for col in df{table} if col.startswith('Blank')]")
        if not blank:
            pass
        else:
            exec(f"df{table}.drop(blank, axis = 1, inplace = True)")

    #full outer join all dataframes
    for x in range(0,9):
        exec(f"df{x}.set_index('Name', inplace = True)")
        #new
        try:
            exec(f"df{x}.drop(droplist, axis=0, inplace=True)")
        except:
            pass
        
    namesnotindf7index = [x for x in df7.index if x not in df0.index]
    namesnotindf8index = [x for x in df8.index if x not in df0.index]
    df7.drop(namesnotindf7index, axis=0, inplace=True)
    df8.drop(namesnotindf8index, axis=0, inplace=True)
    
    result = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8], axis=1, sort = True)
    result.drop('Rk', axis = 1, inplace = True)
    result.reset_index(inplace = True)
    result.rename(columns = {'index': 'Name'}, inplace = True)

    #add TeamName column
    for idx, x in enumerate(result.Name):
        result.loc[idx, 'Team'] = urls[url][44:47]
    
    #add rows to pre-existing dataframe
    all_individuals = pd.concat([all_individuals, result], ignore_index=True)
    
    
    teamstats = pd.read_html(driver.find_element_by_css_selector('table[id="team_and_opponent"]').get_attribute('outerHTML'))[0]

    if 'FG.1' in teamstats.columns:
        ending = '.1'
    else:
        ending = '%'
        
    row = teamstats[teamstats.index == 0].drop('Unnamed: 0', axis=1)
    row.reset_index(drop=True, inplace=True)
    row1 = teamstats[teamstats.index == 1].drop(['Unnamed: 0', 'G', 'FG' + ending, '3P' + ending, '2P' + ending, 'FT' + ending], axis=1)
    row1.reset_index(drop=True, inplace=True)
    row2 = teamstats[teamstats.index == 4].drop('Unnamed: 0', axis=1)
    row2.reset_index(drop=True, inplace=True)
    row3 = teamstats[teamstats.index == 5].drop(['Unnamed: 0', 'G', 'FG' + ending, '3P' + ending, '2P' + ending, 'FT' + ending], axis=1)
    row3.reset_index(drop=True, inplace=True)
    row4 = teamstats[teamstats.index == 3].drop(['Unnamed: 0', 'G'], axis=1)
    row4.reset_index(drop=True, inplace=True)
    row5 = teamstats[teamstats.index == 4].drop(['Unnamed: 0', 'G'], axis=1)
    row5.reset_index(drop=True, inplace=True)

    row.columns = ['G', 'MPTotal', 'FGMadeTotal', 'FGATotal', 'FGPerc', 'ThreeMadeTotal', 'ThreeATotal',
     'ThreePerc', 'TwoMadeTotal', 'TwoATotal', 'TwoPerc', 'FTMadeTotal', 'FTATotal',
     'FTPerc', 'ORBTotal', 'DRBTotal', 'TRBTotal', 'ASTTotal', 'STLTotal','BLKTotal',
     'TOVTotal', 'PFTotal', 'PTSTotal']

    row1.columns = ['MP/G', 'FGMade/G', 'FGA/G', 'ThreeMade/G', 'ThreeA/G',
                     'TwoMade/G', 'TwoA/G', 'FTMade/G', 'FTA/G', 'ORB/G',
                     'DRB/G', 'TRB/G', 'AST/G', 'STL/G','BLK/G', 'TOV/G', 'PF/G', 'PTS/G']

    row2.columns = ['OppG', 'OppMPTotal', 'OppFGMadeTotal', 'OppFGATotal', 'OppFGPerc',
                     'OppThreeMadeTotal', 'OppThreeATotal', 'OppThreePerc', 'OppTwoMadeTotal',
                     'OppTwoATotal', 'OppTwoPerc', 'OppFTMadeTotal', 'OppFTATotal', 'OppFTPerc',
                     'OppORBTotal', 'OppDRBTotal', 'OppTRBTotal', 'OppASTTotal', 'OppSTLTotal',
                     'OppBLKTotal', 'OppTOVTotal', 'OppPFTotal', 'OppPTSTotal']

    row3.columns = ['OppMP/G', 'OppFGMade/G', 'OppFGA/G', 'OppThreeMade/G', 'OppThreeA/G',
                     'OppTwoMade/G', 'OppTwoA/G', 'OppFTMade/G', 'OppFTA/G', 'OppORB/G',
                     'OppDRB/G', 'OppTRB/G', 'OppAST/G', 'OppSTL/G','OppBLK/G', 'OppTOV/G',
                     'OppPF/G', 'OppPTS/G']

    row4.columns = ['MPDiff', 'FGMadeDiff', 'FGADiff', 'FGDiff', 'ThreeMadeDiff', 'ThreeADiff',
                     'ThreePercDiff', 'TwoMadeDiff', 'TwoADiff', 'TwoDiff', 'FTMadeDiff', 'FTADiff',
                     'FTPercDiff', 'ORBDiff', 'DRBDiff', 'TRBDiff', 'ASTDiff', 'STLDiff','BLKDiff',
                     'TOVDiff', 'PFDiff', 'PTSDiff']

    row5.columns = ['OppMPDiff', 'OppFGMadeDiff', 'OppFGADiff', 'OppFGDiff', 'OppThreeMadeDiff', 'OppThreeADiff',
                     'OppThreePercDiff', 'OppTwoMadeDiff', 'OppTwoADiff', 'OppTwoDiff', 'OppFTMadeDiff', 'OppFTADiff',
                     'OppFTPercDiff', 'OppORBDiff', 'OppDRBDiff', 'OppTRBDiff', 'OppASTDiff', 'OppSTLDiff','OppBLKDiff',
                     'OppTOVDiff', 'OppPFDiff', 'OppPTSDiff']

    misc = pd.read_html(driver.find_element_by_css_selector('table[id="team_misc"]').get_attribute('outerHTML'))[0]

    misc.columns = ['Label', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS',
                    'SRS', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr',
                    'OeFGPerc', 'OTOVPerc', 'ORBPerc', 'OFT/FGA', 'DeFGPerc',
                    'DTOVPerc', 'DRBPerc', 'DFT/FGA', 'Arena', 'Attendance']

    row6 = misc[misc.index == 0].drop(['Label', 'Arena', 'Attendance'], axis=1)
    row6.reset_index(drop=True, inplace=True)

    teamstatsrow = pd.concat([row, row1, row2, row3, row4, row5, row6], axis=1, sort=True)
    teamstatsrow.loc[0, 'Team'] = urls[url][44:47]
    
    all_teamstats = pd.concat([all_teamstats, teamstatsrow], ignore_index=True)
    all_teamstats.reset_index(drop=True, inplace=True)
    
driver.quit()

In [7]:
all_individuals.to_pickle("individualstats.pkl")
all_teamstats.to_pickle("1920teamstats.pkl")

In [8]:
all_individuals

,Name,Age,G,GS,MP/G,FGMade/G,FGA/G,FGPerc,ThreeMade/G,ThreeA/G,...,OFoulsC,ShootFoulsD,OFoulsD,PGA,FAAnd1,FABlkd,Salary,Ht,Exp,Team
0,Brook Lopez,31,61,60,26.6,4.0,9.4,0.427,1.4,4.7,...,18,61,3,240,13,20,12093024,84.0,11,MIL
1,D.J. Wilson,23,31,0,9.0,1.3,3.3,0.396,0.5,1.8,...,1,5,3,54,1,3,2961120,82.0,2,MIL
2,Donte DiVincenzo,23,59,22,23.1,3.5,7.7,0.462,1.3,3.7,...,8,30,17,317,2,23,2905800,76.0,1,MIL
3,Dragan Bender,22,7,0,13.0,1.4,3.0,0.476,0.6,1.3,...,0,2,0,21,1,4,1678854,NaN,NaN,MIL
4,Eric Bledsoe,30,56,56,27.2,5.7,11.8,0.482,1.3,3.6,...,23,98,15,733,31,37,15625000,73.0,9,MIL
5,Ersan İlyasova,32,56,8,16.0,2.4,5.2,0.472,0.8,2.2,...,5,31,17,117,7,13,7000000,81.0,11,MIL
6,Frank Mason,25,6,0,8.8,0.8,3.2,0.263,0.3,1.7,...,0,2,0,29,0,3,NaN,NaN,NaN,MIL
7,George Hill,33,52,0,21.2,3.4,6.5,0.530,1.4,2.9,...,4,24,16,361,4,20,9133907,75.0,11,MIL
8,Giannis Antetokounmpo,25,57,57,30.9,10.9,20.0,0.547,1.5,4.8,...,55,276,10,836,86,64,25842697,83.0,6,MIL
9,Khris Middleton,28,55,52,30.1,7.7,15.5,0.499,2.4,5.8,...,19,73,10,519,15,33,30603448,79.0,7,MIL


In [9]:
all_teamstats

,G,MPTotal,FGMadeTotal,FGATotal,FGPerc,ThreeMadeTotal,ThreeATotal,ThreePerc,TwoMadeTotal,TwoATotal,...,3PAr,OeFGPerc,OTOVPerc,ORBPerc,OFT/FGA,DeFGPerc,DTOVPerc,DRBPerc,DFT/FGA,Team
0,65.0,15650,2830,5928,0.477,893,2510,0.356,1937,3418,...,0.423,0.553,12.8,20.7,0.196,0.486,12.2,81.7,0.172,MIL
1,64.0,15460,2596,5665,0.458,880,2369,0.371,1716,3296,...,0.418,0.536,12.8,21.6,0.205,0.502,14.6,76.1,0.200,TOR
2,64.0,15485,2634,5735,0.459,794,2190,0.363,1840,3545,...,0.382,0.529,12.0,23.9,0.204,0.510,13.6,77.5,0.212,BOS
3,65.0,15825,2576,5486,0.470,868,2265,0.383,1708,3221,...,0.413,0.549,13.5,20.5,0.231,0.522,12.3,79.7,0.208,MIA
4,65.0,15700,2740,5744,0.477,648,1787,0.363,2092,3957,...,0.311,0.533,11.9,20.3,0.170,0.513,12.8,77.1,0.193,IND
5,65.0,15675,2651,5701,0.465,742,2052,0.362,1909,3649,...,0.360,0.530,12.7,23.7,0.189,0.522,12.7,80.4,0.211,PHI
6,64.0,15560,2558,5763,0.444,824,2423,0.340,1734,3340,...,0.420,0.515,13.4,23.5,0.199,0.507,10.9,77.8,0.181,BRK
7,65.0,15625,2550,5773,0.442,710,2080,0.341,1840,3693,...,0.360,0.503,11.4,22.4,0.191,0.535,13.5,79.0,0.170,ORL
8,64.0,15435,2682,5821,0.461,788,2117,0.372,1894,3704,...,0.364,0.528,12.1,22.0,0.214,0.560,14.0,74.9,0.230,WAS
9,65.0,15750,2425,5586,0.434,785,2231,0.352,1640,3355,...,0.399,0.504,13.3,23.9,0.188,0.546,13.1,74.4,0.159,CHO


## Scraping NBA Lineups Data from stats.NBA.com

In [7]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")
driver = webdriver.Chrome(executable_path='/Users/Anubhav/Desktop/Flatiron School/QCstats/chromedriver', options=option)

In [8]:
for idx, dftype in enumerate(["traditional", "advanced", "misc", "four-factors", "scoring", "opponent"]):
    exec(f"df{idx} = pd.DataFrame()")
    
    driver.get("https://stats.nba.com/lineups/" + dftype + "/?sort=GROUP_NAME&dir=-1")
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "table")))
    
    page0 = pd.read_html(driver.find_element_by_css_selector('table[class="table"]').get_attribute('outerHTML'))[0]
    
    pages = [page0]
    for pgnum, x in enumerate(range(int(driver.find_element_by_css_selector('div[class="stats-table-pagination__info"]').text[-2:]) - 1)):
        driver.find_element_by_css_selector('a[class="stats-table-pagination__next"]').click()
        exec(f"""page{pgnum+1} = pd.read_html(driver.find_element_by_css_selector('table[class="table"]').get_attribute('outerHTML'))[0]""")
        exec(f"pages.append(page{pgnum+1})")
    
    exec(f"df{idx} = pd.concat(pages, ignore_index=True)")

driver.quit()

In [12]:
#drop uncessary columns from each dataframe
for table in range(1,6):
    checklist = []
    for x in range(0,table):
        exec(f"checklist.extend(list(df{x}.columns))")
        checklist.remove('Lineups')
    exec(f"df{table}.drop([col for col in df{table}.columns if col in checklist], axis = 1, inplace = True)")

#full outer join all dataframes
for x in range(0,6):
    exec(f"df{x}.set_index('Lineups', inplace = True)")

lineupdf = pd.concat([df0, df1, df2, df3, df4, df5], axis=1, sort = True)
lineupdf.reset_index(inplace = True)
lineupdf.rename(columns = {'index': 'Lineups'}, inplace = True)

In [15]:
lineupdf.to_pickle("lineupdata.pkl")